In [1]:
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
# Класс, выполняющий стандартизацию с реализацией корректировки на случай присутствия бинарных переменных 
# (когда делим не на одно, а на два стандартных отклонения).

class StandardScalerFF(TransformerMixin):
    def __init__(self, copy=True):
        self.copy = copy
        
    def fit(self, X, y=None):
        # Создаём копию, чтобы не менять исходный датафрейм
        if self.copy == True:
            X = X.copy()

        # Проверяем тип данных входного массива.
        # Если не DataFrame, то преобразуем в него
        if isinstance(X, pd.DataFrame) == False:
            X = pd.DataFrame(X)
            X = X.astype(object).replace("None", np.nan)
            X = X.astype(object).replace("nan", np.nan)   
            
        self.num_columns = [] # Список для вещественных признаков
        self.bin_columns = [] # Список для бинарных переменных
        for col in X.dtypes[X.dtypes != "object"].index:
            if X[col].nunique() > 2:
                self.num_columns.append(col)
            else:
                self.bin_columns.append(col)
        
        # считаем mean и std для вещественных признаков
        self.X_num_mean = X[self.num_columns].mean()
        self.X_num_std = X[self.num_columns].std()
        
        # считаем mean и std для бинарных признаков
        self.X_bin_mean = X[self.bin_columns].mean()
        self.X_bin_std = X[self.bin_columns].std()
        
        return self

    def transform(self, X):
        # Создаём копию, чтобы не менять исходный датафрейм
        if self.copy == True:
            X = X.copy()
            
        # Проверяем тип данных входного массива.
        # Если не DataFrame, то преобразуем в него
        if isinstance(X, pd.DataFrame) == False:
            X = pd.DataFrame(X)
            X = X.astype(object).replace("None", np.nan)
            X = X.astype(object).replace("nan", np.nan)
            
        X[self.num_columns] = (X[self.num_columns] - self.X_num_mean) / self.X_num_std
        X[self.bin_columns] = (X[self.bin_columns] - self.X_bin_mean) / (2 * self.X_bin_std)
        
        return X

In [3]:
toy_train = pd.DataFrame({'Balance': [8.3, 9.4, 10.2, 0],
'Age': [23, 29, 36, 44]})
toy_train

,Balance,Age
0,8.3,23
1,9.4,29
2,10.2,36
3,0.0,44


In [4]:
toy_test = pd.DataFrame({'Balance': [10.4, 3.1, 22.5, -1],
'Age': [13, 19, 66, 33]})
toy_test

,Balance,Age
0,10.4,13
1,3.1,19
2,22.5,66
3,-1.0,33


In [5]:
ssff = StandardScalerFF()

In [6]:
ssff.fit(toy_train)

In [7]:
ssff.fit(toy_train)
ssff.transform(toy_train)

,Balance,Age
0,0.281031,-1.104315
1,0.514340,-0.441726
2,0.684019,0.331295
3,-1.479390,1.214747


In [8]:
ssff.X_num_mean

Balance     6.975
Age        33.000
dtype: float64

In [9]:
ssff.X_num_std

Balance    4.714782
Age        9.055385
dtype: float64

In [10]:
ssff.transform(toy_test)

,Balance,Age
0,0.726439,-2.208631
1,-0.821883,-1.546041
2,3.292835,3.644240
3,-1.691489,0.000000
